In [1]:
#Import des librairies nécessaires

import pandas as pd
import numpy as np
import nltk

from nltk.corpus import stopwords
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel 
from sklearn.metrics.pairwise import cosine_similarity
import operator
from sklearn.neighbors import NearestNeighbors

# Concaténation des Dataframes

Ici, on importe les différents Dataframes créés tout au long du scrapping, puis on les concatène tous les trois DataFrames pour créer différents datasets intermédiaires utiles en cas de tests. 

In [2]:
path="C:/Users/Utilisateur/Dropbox/Formation professionnelle IA/Projet Final/dataset final/"
df1=pd.read_csv(path+"final1.csv")
df2=pd.read_csv(path+"final2.csv")
df3,df4=pd.read_csv(path+"final3.csv"),pd.read_csv(path+"final4.csv")
df5,df6=pd.read_csv(path+"final5.csv"),pd.read_csv(path+"final6.csv")
df7,df8=pd.read_csv(path+"final7.csv"),pd.read_csv(path+"final8.csv")
df9,df10=pd.read_csv(path+"final9.csv"),pd.read_csv(path+"final10.csv")
df11,df12=pd.read_csv(path+"final11.csv"),pd.read_csv(path+"final12.csv")
df13,df14,df15=pd.read_csv(path+"final13.csv"),pd.read_csv(path+"final14.csv"),pd.read_csv(path+"final15.csv")
df16,df17,df18=pd.read_csv(path+"final16.csv"),pd.read_csv(path+"final17.csv"),pd.read_csv(path+"final18.csv")

df123=pd.concat([df1,df2,df3],ignore_index=True)
df456=pd.concat([df4,df5,df6],ignore_index=True)
df789=pd.concat([df7,df8,df9],ignore_index=True)
df101112=pd.concat([df10,df11,df12],ignore_index=True)
df131415=pd.concat([df13,df14,df15],ignore_index=True)
df161718=pd.concat([df16,df17,df18],ignore_index=True)

In [3]:
#Cette cellule permet de calculer le nombre de groupe scrappés jusqu'à présent. 

nombre=df1.shape[0]+df2.shape[0]+df3.shape[0]+df4.shape[0]+df5.shape[0]+df6.shape[0]+df7.shape[0]

nombre2=df8.shape[0]+df9.shape[0]+df10.shape[0]+df11.shape[0]+df12.shape[0]+df13.shape[0]+df14.shape[0]
nombre3=df15.shape[0]+df16.shape[0]+df17.shape[0]+df18.shape[0]
somme=nombre+nombre2+nombre3
print("nombre de groupes scrappés :",somme)

nombre de groupes scrappés : 683


# Vérification scraping 

Cette partie permet de récupérer les groupes nécessitant une vérification. En effet, en raison de la présence d'homonymes, parfois, les groupes récupérés par le scraping ne correspondent pas aux groupes recherchés initialement.

Par conséquent, étant donné que les groupes de Métal réunissant plus de 100 favoris sont rares, on récupére tous les groupes au-delà de cette limite. Par la suite, on récupérera les URL des groupes que l'on souhaite obtenir et on refera un scraping individuel de façon à récupérer les valeurs individuelles pour chaque groupe. Pour éviter les erreurs, on excluera dans un premier temps les groupes de cette liste dans le programme final de recommandation. Une fois le scraping corrigé, ils seront réintégrés en bonne et due forme. 

In [4]:
erreur=list(df123[df123['favoris']>=100]['nom'])
erreur

['30 Seconds To Mars',
 'A Different Breed Of Killer',
 'A Perfect Circle',
 'AC-DC',
 'Aerosmith',
 'Agalloch',
 'Airbourne',
 'Alcest',
 'Alestorm',
 'Alice Cooper',
 'Amon Amarth',
 'Anthrax',
 'Apocalyptica']

In [5]:
#Fonction permettant de rescrapper individuellement les groupes faux directement à partir de l'URL du groupe visé.


def alternative_spirit(elem,lien):
    notes=[]
    driver = webdriver.Firefox()
    driver.get(lien)
    favoris=driver.find_element_by_class_name("cvi-totallike").get_attribute("data-sc-like-count")
    b=driver.find_elements_by_class_name("erra-global")
    c=driver.find_elements_by_class_name("erra-ratings")
    for i in c:
        b=i.find_element_by_tag_name("a")
        if b.text!='-':
            notes.append(float(b.text))
    moy=round(np.mean(notes),2)
    time.sleep(2)
    driver.quit()
    return elem,moy,favoris

elem=list(df123[df123["nom"]==erreur[0]]["nom"])[0]
notes=[]
noms=[]
lien="https://www.senscritique.com/contact/Thirty_Seconds_to_Mars/577641" 
#res=alternative_spirit(elem,lien)


# Preprocessing

In [6]:
'''
Remplacement des valeurs nulles par la valeur "unknown" pour permettre un meilleur traitement par l'algorithme par la suite. 
'''

df123['lyrical themes']=df123['lyrical themes'].fillna("unknown")

In [7]:
#Vérification des valeurs manquantes
df123.isna().sum()

nom               0
pays              0
statut            0
genre             0
years             0
lyrical themes    0
note moyenne      0
favoris           0
dtype: int64

Téléchargement des imports nécessaires au nettoyage du texte 

In [8]:

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Utilisateur\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Utilisateur\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
'''
Fonction de nettoyage du texte : suppression des accents, des éléments autre que les caractères et les chiffres,
suppression des stop_words. 
'''




stop = stopwords.words('english')
stop_words_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()

def black_txt(token):
    return  token not in stop_words_ and token not in list(string.punctuation)  and len(token)>2   
  
def clean_txt(text):
    clean_text = []
    clean_text2 = []
    text = re.sub("'", "",text)
    text=re.sub("(\\d|\\W)+"," ",text) 
    clean_text = [ wn.lemmatize(word, pos="v") for word in word_tokenize(text.lower()) if black_txt(word)]
    clean_text2 = [word for word in clean_text if black_txt(word)]
    return " ".join(clean_text2)


In [11]:
'''
Par souci d'éviter de modifier le dataset original, on réalisera les prochaines opérations sur une copie du
Dataframe original. 
'''

df2=df123.copy()

On décidera ici de tester différents algorithmes. : 

- TF-IDF Vectorizer associé à la cosine similarity du module linear_kernel. Toutes ces techniques sont tirées de la librairie scikit-learn. 

- TF-IDF Vectorizer associé à l'algorithme KNN et à la cosine similarity du module linear kernel. 

- CountVectorizer associé à la cosine_similarity du module pairwise

- Count Vectorizer associé à la correlation de Pearson. 

Pour chaque technique, on appliquera les algorithmes d'abord sur la colonne des paroles, puis sur la colonne genre. Enfin, on mettra en commun les résultats obtenus avec les deux colonnes en cherchant à faire ressortir les groupes recommandés dans les deux cas. 


# TF-IDF Vectorizer

## Colonne paroles

In [12]:
"""
Ici, on crée une copie du DataFrame qui ne contient que les groupes pour lesquelles on a des infos sur les paroles.
En effet, cela n'est pas utile de lancer un algorithme de NLP sur des colonnes sans contenu à traiter. 
"""

lyrics=pd.DataFrame(df2[df2["lyrical themes"]!="unknown"],columns=["nom","lyrical themes"])
lyrics=lyrics.reset_index(drop=True)
lyrics

,nom,lyrical themes
0,1349,"Anti-religion, Apocalypse, Darkness"
1,Aäkon Këëtrëh,"Despair, Loneliness, Winter feelings"
2,Aaskereia,"Darkness, Winter, Forests"
3,Abbath,"War, Winter"
4,Abigail Williams,"Metaphysics, Spiritualism, Occultism, Nature"
...,...,...
76,Anvil,"Love, Heavy Metal, Sex, Life"
77,Aosoth,"Satanism, Death, Darkness"
78,Apocalyptica,"Mostly instrumental, Love, Relationship issues"
79,Arafel,"Legends, Fantasy, Mysticism, Abstract themes, ..."


In [13]:
"""
Instanciation de l'algorithme sans preprocessing. En effet, l'algorithme TF-IDF Vectorizer contient 
tous les éléments nécessaires au preprocessing. Par conséquent, cela n'est pas forcément utile de réaliser un
preprocessing du texte avant d'intégrer les données à l'algorithme. 
"""

tf = TfidfVectorizer(analyzer='word', min_df=0,stop_words='english')

In [14]:
#Intégration des données à l'algorithme 

tfidf_matrix_l = tf.fit_transform(lyrics['lyrical themes'])
tfidf_matrix_l.shape

(81, 165)

In [15]:
"""
Application de la cosine similarity et récupération des résultats de cosine similarity pour chaque groupe 
dans un dictionnaire trié selon les valeurs de cosine similarity en récupérant les noms des groupes au passage. 
"""
cosine_similarities = linear_kernel(tfidf_matrix_l, tfidf_matrix_l) 
results = {}
for idx, row in lyrics.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[-100:-1] 
    similar_items = [(cosine_similarities[idx][i], lyrics['nom'][i]) for i in similar_indices]
    results[row['nom']] = similar_items[1:]

In [16]:
#Affichage du dictionnaire obtenu donnant les cosine similarity pour chaque groupe. 
results

{'1349': [(0.0, 'Aktarum'),
  (0.0, 'Alcatrazz'),
  (0.0, 'Aldaaron'),
  (0.0, 'Arafel'),
  (0.0, 'Alkonost'),
  (0.0, 'All Shall Perish'),
  (0.0, 'All That Remains'),
  (0.0, 'Allegaeon'),
  (0.0, 'Almah'),
  (0.0, 'Amberian Dawn'),
  (0.0, 'Amenra'),
  (0.0, 'Amesoeurs'),
  (0.0, 'Amon Amarth'),
  (0.0, 'Amoral'),
  (0.0, 'Amorphis'),
  (0.0, 'Anaal Nathrakh'),
  (0.0, 'Anal Blast'),
  (0.0, 'Apocalyptica'),
  (0.0, 'Anvil'),
  (0.0, 'Antestor'),
  (0.0, 'Antaeus'),
  (0.0, 'Anorexia Nervosa'),
  (0.0, 'Annihilator'),
  (0.0, 'Akercocke'),
  (0.0, 'Annatar'),
  (0.0, 'Angel Witch'),
  (0.0, 'Andre Matos'),
  (0.0, 'And Hell Followed With'),
  (0.0, 'Ancient Rites'),
  (0.0, 'Ancient Bards'),
  (0.0, 'Anal Vomit'),
  (0.0, 'Angelus Apatrida'),
  (0.0, 'Ajattara'),
  (0.0, 'Arcana'),
  (0.0, 'Agoraphobic Nosebleed'),
  (0.0, 'Aäkon Këëtrëh'),
  (0.0, 'Abbath'),
  (0.0, 'Abigail Williams'),
  (0.0, 'Abominable Putridity'),
  (0.0, 'Aborted'),
  (0.0, 'Aborym'),
  (0.0, 'Absu'),
  (0.0,

In [17]:
"""
Création d'une fonction  permettant de récupérer les meilleurs recommandations pour un groupe donné. On 
applique dans la fonction un filtre permettant de récupérer les meilleurs résultats à partir d'une certaine 
valeur de cosine similarity. 
"""

def recommandation_paroles(name,filtre):
    best_bands=[]
    best_scores=[]
    #print("Je vous recommande les groupes suivants:")
    for i in results[name]:
        if i[0]>filtre:
            best_bands.append(i[1])
            best_scores.append(i[0])
    best_bands=best_bands[::-1]
    best_scores=best_scores[::-1]
    return best_bands,best_scores
    

In [18]:
# Affichage des résultats obtenus (groupe et similarité cosinus) pour le groupe "Amberian Dawn". 

resultat1=recommandation_paroles("Amberian Dawn",0.3)
resultat1

(['Alkonost', 'Ancient Bards', 'Annatar', 'Arafel', 'Altaria', 'Ajattara'],
 [0.5020352283304641,
  0.46742137155484487,
  0.46742137155484487,
  0.31573363302221696,
  0.3146627358452062,
  0.30328307621846273])

In [19]:
"""
Vérification de la recommandation proposée par l'algorithme pour le groupe test et le groupe le plus similaire
selon l'algorithme. On retrouve bien des thèmes de paroles similaires dans les deux cas. L'algorithme est 
donc efficace. 

"""
ab=list(lyrics[lyrics["nom"]=='Alkonost']["lyrical themes"])
cd=list(lyrics[lyrics["nom"]=='Amberian Dawn']["lyrical themes"])
ab,cd

(['Fantasy, Folklore, Paganism'], ['Fantasy, Paganism, Kalevala'])

## Colonne genre

On réalise la même procédure que précédemment pour la colonne contenant les genres de Métal. 

In [20]:
#Création d'une copie du DataFrame ne contenant que les noms et les genres pour chaque groupe. 

genre=pd.DataFrame(df2,columns=["nom","genre"]).reset_index(drop=True)
genre


,nom,genre
0,12 Stones,Metal Alternatif
1,12012,Visual Kei
2,1349,Black Metal
3,25 Ta Life,Hardcore
4,30 Seconds To Mars,Metal Alternatif
...,...,...
118,Aosoth,Black Metal
119,Apocalyptica,Symphonic Heavy Metal/Rock
120,AqME,Metal Alternatif
121,Arafel,Folk Black Metal


In [21]:

tf = TfidfVectorizer(analyzer='word', min_df=0,stop_words='english')
tfidf_matrix_g = tf.fit_transform(genre['genre'])
cosine_similarities = linear_kernel(tfidf_matrix_g, tfidf_matrix_g) 
results2 = {}
for idx, row in genre.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[-100:-1] 
    similar_items = [(cosine_similarities[idx][i], genre['nom'][i]) for i in similar_indices]
    results2[row['nom']] = similar_items[1:]


In [22]:
results2

{'12 Stones': [(0.0, 'A Day To Remember'),
  (0.0, 'A Static Lullaby'),
  (0.0, 'A Skylit Drive'),
  (0.030255656762029178, 'All That Remains'),
  (0.04258892896358387, 'Amesoeurs'),
  (0.05003587059493216, 'Amebix'),
  (0.05196409116717673, 'Abominable Putridity'),
  (0.053809083743907374, 'Alkonost'),
  (0.055574347599078894, 'Adrian Von Ziegler'),
  (0.05612545839798782, 'Agoraphobic Nosebleed'),
  (0.0566504771557351, 'Altaria'),
  (0.05754406757218926, 'Acid Witch'),
  (0.05798522894524815, 'Amenra'),
  (0.05817117208695154, 'Aes Dana'),
  (0.05825943874876092, 'Anaal Nathrakh'),
  (0.05863239129566635, 'Amberian Dawn'),
  (0.058700570024331225, 'Adrenaline Mob'),
  (0.0610139097846348, 'Agent Steel'),
  (0.0614791153437053, 'Angel Witch'),
  (0.0614791153437053, 'Alcatrazz'),
  (0.061608236744855056, 'And Hell Followed With'),
  (0.06256828338526933, 'Ahab'),
  (0.06256828338526933, 'Acid King'),
  (0.06256828338526933, 'Acid Drinkers'),
  (0.06311866625070524, 'Allegaeon'),
  (0

In [23]:
def recommandation_genres(name,filtre):
    best_bands=[]
    best_scores=[]
    #print("Je vous recommande les groupes suivants:")
    for i in results2[name]:
        if i[0]>filtre:
            best_bands.append(i[1])
            best_scores.append(i[0])
    best_bands=best_bands[::-1]
    best_scores=best_scores[::-1]
    '''
    for i in range(len(best_bands)):
        for j in range(len(best_scores)):
            if i==j:
                print(best_bands[i], "avec un score de :",best_scores[j])
    '''
    return best_bands,best_scores


In [24]:
#Affihage des résultats obtenus pour le groupe retenu pour tous les tests : "Amberian Dawn". 

resultat2=recommandation_genres("Amberian Dawn",0.3)
resultat2

(['Arcana',
  'Altaria',
  'Ancient Bards',
  'Alestorm',
  'Almah',
  'Andre Matos',
  'Anvil',
  'Ace Frehley',
  'AC-DC',
  'Airbourne',
  'Aerosmith',
  'Alice Cooper',
  'Angra'],
 [0.49748668349693803,
  0.4879433263597296,
  0.37057893138467557,
  0.36554690540170676,
  0.36554690540170676,
  0.36006527944662303,
  0.36006527944662303,
  0.3241903178647788,
  0.3241903178647788,
  0.3241903178647788,
  0.3241903178647788,
  0.3241903178647788,
  0.3209624107350458])

## Mise en commun des résultats 

Dans cette partie, on affiche les résultats obtenus pour les deux colonnes. Ensuite, on cherche à afficher dans une liste les groupes communs aux recommandations pour chacune des deux colonnes. Enfin, on vérifie les résultats obtenus et on constate que les résultats obtenus permettent de mesurer l'efficacité de cette méthode pour faire de la recommandation basée sur du NLP. 

In [25]:
resultat1,resultat2

((['Alkonost', 'Ancient Bards', 'Annatar', 'Arafel', 'Altaria', 'Ajattara'],
  [0.5020352283304641,
   0.46742137155484487,
   0.46742137155484487,
   0.31573363302221696,
   0.3146627358452062,
   0.30328307621846273]),
 (['Arcana',
   'Altaria',
   'Ancient Bards',
   'Alestorm',
   'Almah',
   'Andre Matos',
   'Anvil',
   'Ace Frehley',
   'AC-DC',
   'Airbourne',
   'Aerosmith',
   'Alice Cooper',
   'Angra'],
  [0.49748668349693803,
   0.4879433263597296,
   0.37057893138467557,
   0.36554690540170676,
   0.36554690540170676,
   0.36006527944662303,
   0.36006527944662303,
   0.3241903178647788,
   0.3241903178647788,
   0.3241903178647788,
   0.3241903178647788,
   0.3241903178647788,
   0.3209624107350458]))

In [26]:
res1=recommandation_genres("Amberian Dawn",0.3)[0]
res2=recommandation_paroles("Amberian Dawn",0.3)[0]
liste_commune=[i for i in res2 for j in res1 if i==j]
liste_commune

['Ancient Bards', 'Altaria']

In [27]:
a=list(df2[df2["nom"]=="Amberian Dawn"]["genre"])
b=list(df2[df2["nom"]=="Ancient Bards"]["genre"])
print(a,b)

['Neoclassical Power Metal, Rock'] ['Symphonic Power Metal']


In [28]:
c=list(df2[df2["nom"]=="Amberian Dawn"]["lyrical themes"])
d=list(df2[df2["nom"]=="Ancient Bards"]["lyrical themes"])
print(c,d)

['Fantasy, Paganism, Kalevala'] ['Fantasy']


# CountVectorizer 

## Colonne lyrics

In [29]:
#Récupération du DataFrame créé lors de la méthode précédente.

lyrics

,nom,lyrical themes
0,1349,"Anti-religion, Apocalypse, Darkness"
1,Aäkon Këëtrëh,"Despair, Loneliness, Winter feelings"
2,Aaskereia,"Darkness, Winter, Forests"
3,Abbath,"War, Winter"
4,Abigail Williams,"Metaphysics, Spiritualism, Occultism, Nature"
...,...,...
76,Anvil,"Love, Heavy Metal, Sex, Life"
77,Aosoth,"Satanism, Death, Darkness"
78,Apocalyptica,"Mostly instrumental, Love, Relationship issues"
79,Arafel,"Legends, Fantasy, Mysticism, Abstract themes, ..."


In [30]:
# Création d'une copie du DataFrame lyrics pour ne pas altérer son contenu. 


lyrics2=lyrics.copy()

In [31]:
#Preprocessing de la colonne contenant les données à intégrer à l'algorithme. 

lyrics2["lyrical themes"]=lyrics2["lyrical themes"].apply(clean_txt)

In [32]:
#Instanciation de l'algorithme et intégration des données à l'intérieur de celui-ci. 

vectorizer = CountVectorizer()
X_lyrics= vectorizer.fit_transform((lyrics2['lyrical themes']))

#counts = pd.DataFrame(X_lyrics.toarray(),columns=vectorizer.get_feature_names())
#counts

In [33]:
#Applicartion de la similarité cosinus sur la matrice issue de l'algorithme. 

cosine_sim = cosine_similarity(X_lyrics)

In [34]:
"""

Création d'une fonction qui permet de récupérer pour chaque groupe les meilleurs scores de chaque groupe dans un
dictionnaire, en associant chaque score au groupe concerné. Puis on trie le dictionnaire selon les meilleurs scores
obtenus pour chaque groupe. Puis, selon le nombre de recommandations n souhaités, on affiche les n 
meilleurs scores et groupes associés pour le groupe donné en entrée. 

"""



def meilleurs_reco_lyrics(elem,nombre_de_reco):
    elem_index=lyrics2[lyrics2["nom"]==elem].index
    similar_movies = list(cosine_sim[elem_index][0])
    a=lyrics2["nom"]
    b=similar_movies
    sim={a[i]:b[j] for i in range(len(a)) for j in range(len(b)) if i==j}
    sortedDict = sorted(sim.items(), key=operator.itemgetter(1),reverse=True)
    sortedDict=[i for i in sortedDict if i[1]!=0]
    return sortedDict[1:nombre_de_reco+1]

In [35]:
#Affichage des résultats obtenus pour le groupe test : Amberian Dawn. 

elem="Amberian Dawn"
n=10
resultat3=meilleurs_reco_lyrics(elem,n)
resultat3

[('Alkonost', 0.6666666666666669),
 ('Ancient Bards', 0.5773502691896258),
 ('Annatar', 0.5773502691896258),
 ('Arafel', 0.4364357804719848),
 ('Aktarum', 0.408248290463863),
 ('Altaria', 0.408248290463863),
 ('Adorned Brood', 0.2886751345948129),
 ('Ajattara', 0.2886751345948129),
 ('Aldaaron', 0.2886751345948129),
 ('Angel Witch', 0.25819888974716115)]

In [36]:
"""
Vérification des résultats obtenus en affichant les données pour le groupe test et la meilleure recommandation
faite par l'algorithme. On constate une fois de plus l'efficacité remarquable de l'algorithme. 
"""

lyrics2[lyrics2["nom"]==elem]["lyrical themes"]

48    fantasy paganism kalevala
Name: lyrical themes, dtype: object

In [37]:
elem2=resultat3[0][0]
lyrics2[lyrics2["nom"]==elem2]["lyrical themes"]

42    fantasy folklore paganism
Name: lyrical themes, dtype: object

## Colonne genre 
On applique ici l'algorithme sur la colonne des genres en reprenant la même méthode que précédemment. Puis on affiche comme pour le cas précédent les résultats et on vérifie l'efficacité de l'algorithme. On pourra constater une fois de plus son efficacité sur cette colonne comme précedemment. 

In [38]:
genre

,nom,genre
0,12 Stones,Metal Alternatif
1,12012,Visual Kei
2,1349,Black Metal
3,25 Ta Life,Hardcore
4,30 Seconds To Mars,Metal Alternatif
...,...,...
118,Aosoth,Black Metal
119,Apocalyptica,Symphonic Heavy Metal/Rock
120,AqME,Metal Alternatif
121,Arafel,Folk Black Metal


In [39]:
genre2=genre.copy()

In [40]:
genre2["genre"]=genre2["genre"].apply(clean_txt)

In [41]:
vectorizer = CountVectorizer()
X_genre= vectorizer.fit_transform((genre2['genre']))

In [42]:
cosine_sim = cosine_similarity(X_genre)

In [43]:

def meilleurs_reco_genre(elem,nombre_de_reco):
    elem_index=genre2[genre2["nom"]==elem].index
    similar_movies = list(cosine_sim[elem_index][0])
    a=genre2["nom"]
    b=similar_movies
    sim={a[i]:b[j] for i in range(len(a)) for j in range(len(b)) if i==j}
    sortedDict = sorted(sim.items(), key=operator.itemgetter(1),reverse=True)
    sortedDict=[i for i in sortedDict if i[1]!=0]
    return sortedDict[1:nombre_de_reco+1]

In [44]:
elem="Amberian Dawn"
n=10
resultat4=meilleurs_reco_genre(elem,n)
resultat4

[('Altaria', 0.6708203932499369),
 ('Alestorm', 0.5773502691896258),
 ('Almah', 0.5773502691896258),
 ('Ancient Bards', 0.5773502691896258),
 ('Andre Matos', 0.5773502691896258),
 ('Angra', 0.5773502691896258),
 ('Anvil', 0.5773502691896258),
 ('Adrenaline Mob', 0.5),
 ('Agalloch', 0.5),
 ('Agent Steel', 0.5)]

In [45]:
genre2[genre2["nom"]==elem]["genre"]

82    neoclassical power metal rock
Name: genre, dtype: object

In [46]:
genre2[genre2["nom"]==resultat4[0][0]]["genre"]

78    melodic power metal hard rock
Name: genre, dtype: object

## Mise en commun des résultats
Comme pour l'algorithme testé dans la partie précédente, on affiche les résultats obtenus et on affiche les recommandations communes pour les deux colonnes. 

In [47]:
resultat3,resultat4

([('Alkonost', 0.6666666666666669),
  ('Ancient Bards', 0.5773502691896258),
  ('Annatar', 0.5773502691896258),
  ('Arafel', 0.4364357804719848),
  ('Aktarum', 0.408248290463863),
  ('Altaria', 0.408248290463863),
  ('Adorned Brood', 0.2886751345948129),
  ('Ajattara', 0.2886751345948129),
  ('Aldaaron', 0.2886751345948129),
  ('Angel Witch', 0.25819888974716115)],
 [('Altaria', 0.6708203932499369),
  ('Alestorm', 0.5773502691896258),
  ('Almah', 0.5773502691896258),
  ('Ancient Bards', 0.5773502691896258),
  ('Andre Matos', 0.5773502691896258),
  ('Angra', 0.5773502691896258),
  ('Anvil', 0.5773502691896258),
  ('Adrenaline Mob', 0.5),
  ('Agalloch', 0.5),
  ('Agent Steel', 0.5)])

In [48]:
elem="Amberian Dawn"
n=10
res=meilleurs_reco_lyrics(elem,n)
res2=meilleurs_reco_genre(elem,n)

In [49]:
liste1=[resultat3[i][0] for i in range(len(resultat3))]
liste2=[resultat4[i][0] for i in range(len(resultat4))]
commun=[i for i in liste1 for j in liste2 if i==j]
print(commun)

['Ancient Bards', 'Altaria']


### Test de l'algorithme K-Nearest Neighbors 

## Paroles

In [50]:
#Reprise du Dataframe pour cette partie comme pour les parties précédentes. 

lyrics

,nom,lyrical themes
0,1349,"Anti-religion, Apocalypse, Darkness"
1,Aäkon Këëtrëh,"Despair, Loneliness, Winter feelings"
2,Aaskereia,"Darkness, Winter, Forests"
3,Abbath,"War, Winter"
4,Abigail Williams,"Metaphysics, Spiritualism, Occultism, Nature"
...,...,...
76,Anvil,"Love, Heavy Metal, Sex, Life"
77,Aosoth,"Satanism, Death, Darkness"
78,Apocalyptica,"Mostly instrumental, Love, Relationship issues"
79,Arafel,"Legends, Fantasy, Mysticism, Abstract themes, ..."


In [51]:
lyrics2=lyrics.copy()
lyrics2["lyrical themes"]=lyrics2["lyrical themes"].apply(clean_txt)

In [52]:
lyrics2[:30]

,nom,lyrical themes
0,1349,anti religion apocalypse darkness
1,Aäkon Këëtrëh,despair loneliness winter feel
2,Aaskereia,darkness winter forest
3,Abbath,war winter
4,Abigail Williams,metaphysics spiritualism occultism nature
5,Abigor,satanism darkness death
6,Abominable Putridity,gore conspiracies genetic mutation biology hor...
7,Aborted,death gore violence psychopathy
8,Aborym,occultism satanism spiritualism esotericism
9,Abruptum,satanism evil darkness torture death pain


In [53]:
"""
Instanciation de l'algorithme TF-IDF Vectorizer en premier lieu pour vectoriser les données à implanter 
dans l'algorithme.
"""
tf = TfidfVectorizer(stop_words='english')
X = tf.fit_transform(lyrics2["lyrical themes"])

In [54]:

"""
Création d'une fonction permettant d'afficher les recommandations proposées par l'algorithme KNN. Le nombre
de recommandation correspond au nombre de plus proches voisins que l'algorithme doit détecter. Puis comme pour
les parties précédentes, on récupère les meilleurs résultats pour le groupe donné en entrée dans un 
dictionnaire en associant les noms de groupes aux résultats. Puis on affiche les résultats obtenus. 
"""

def reco_KNN(elem,nbre_de_reco):
    nbre_de_reco
    KNN = NearestNeighbors(nbre_de_reco+1, p=2)
    KNN.fit(X)
    test=lyrics2[lyrics2["nom"]==elem]
    matrix=tf.transform(test["lyrical themes"])
    voisins=KNN.kneighbors(matrix,return_distance=True)
    index_groupes=voisins[1][0][1:]
    distance=voisins[0][0][1:]
    liste_groupes=[lyrics2["nom"][index_groupes[i]] for i in range(len(index_groupes))]
    res=[(liste_groupes[i],distance[j]) for i in range(len(liste_groupes)) for j in range(len(distance)) if i==j]
    return res

In [55]:
"""
Affichge des résultats pour le groupe test. On obtient les distances entre le point du groupe d'entrée 
et les groupes les plus proches. 
"""
elem="Amberian Dawn"
n=5
resultat5=reco_KNN(elem,n)

In [56]:
resultat5

[('Alkonost', 0.9979626963664883),
 ('Ancient Bards', 1.0320645604274523),
 ('Annatar', 1.0320645604274523),
 ('Arafel', 1.1698430381703206),
 ('Altaria', 1.1707580998265985)]


Conversion des distances en pourcentage pour afficher la contribution de chaque groupe dans la distance cumulée
par rappot au groupe de référence pour obtenir plus de lisibilité dans le résultat et le comparer à la
similarité cosinus. 


In [57]:
total=float(resultat5[0][1])+float(resultat5[1][1])+float(resultat5[2][1])+float(resultat5[3][1])+float(resultat5[4][1])

res1,res2,res3,res4,res5=(1-(float(resultat5[0][1])/total)),(1-(float(resultat5[1][1])/total)),(1-(float(resultat5[2][1])/total)),(1-(float(resultat5[3][1])/total)),(1-(float(resultat5[4][1])/total))
liste3=[res1,res2,res3,res4,res5]
liste3

[0.8152842101821494,
 0.8089721979423965,
 0.8089721979423965,
 0.7834703826652962,
 0.7833010112677613]

## Genre

On applique la même méthode que pour la colonne des paroles. 

In [58]:
genre

,nom,genre
0,12 Stones,Metal Alternatif
1,12012,Visual Kei
2,1349,Black Metal
3,25 Ta Life,Hardcore
4,30 Seconds To Mars,Metal Alternatif
...,...,...
118,Aosoth,Black Metal
119,Apocalyptica,Symphonic Heavy Metal/Rock
120,AqME,Metal Alternatif
121,Arafel,Folk Black Metal


In [59]:
genre2=genre.copy()
genre2["genre"]=genre2["genre"].apply(clean_txt)

In [60]:
tf = TfidfVectorizer(stop_words='english')
X = tf.fit_transform(genre2["genre"])

In [61]:
def reco_KNN_genre(elem,nbre_de_reco):
    nbre_de_reco
    KNN = NearestNeighbors(nbre_de_reco+1, p=2)
    KNN.fit(X)
    test=genre2[genre2["nom"]==elem]
    matrix=tf.transform(test["genre"])
    voisins=KNN.kneighbors(matrix,return_distance=True)
    index_groupes=voisins[1][0][1:]
    distance=voisins[0][0][1:]
    liste_groupes=[genre2["nom"][index_groupes[i]] for i in range(len(index_groupes))]
    res=[(liste_groupes[i],distance[j]) for i in range(len(liste_groupes)) for j in range(len(distance)) if i==j]
    return res

In [62]:
elem="Amberian Dawn"
n=5
resultat6=reco_KNN_genre(elem,n)
resultat6

[('Arcana', 1.002510166036297),
 ('Altaria', 1.0119848552624395),
 ('Ancient Bards', 1.121981344421844),
 ('Almah', 1.1264573623518053),
 ('Alestorm', 1.1264573623518053)]

In [63]:
total=float(resultat6[0][1])+float(resultat6[1][1])+float(resultat6[2][1])+float(resultat5[3][1])+float(resultat6[4][1])
res5,res6,res7,res8,res9=(1-(float(resultat6[0][1])/total)),(1-(float(resultat6[1][1])/total)),(1-(float(resultat6[2][1])/total)),(1-(float(resultat6[3][1])/total)),(1-(float(resultat6[4][1])/total))
liste4=[res5,res6,res7,res8,res9]
liste4

[0.8154700240463533,
 0.8137260375669133,
 0.7934792109638248,
 0.7926553195869926,
 0.7926553195869926]

## Mise en commun des résultats
Même procédure que la partie précédente : affichage des résultats pour les deux colonnes, affichage des groupes communs aux deux recommandations. 

In [64]:
liste_groupes=[resultat6[i][0] for i in range(len(resultat6))]
liste_groupes

['Arcana', 'Altaria', 'Ancient Bards', 'Almah', 'Alestorm']

In [65]:
liste3,liste4

([0.8152842101821494,
  0.8089721979423965,
  0.8089721979423965,
  0.7834703826652962,
  0.7833010112677613],
 [0.8154700240463533,
  0.8137260375669133,
  0.7934792109638248,
  0.7926553195869926,
  0.7926553195869926])

In [66]:
liste_groupes2=[resultat5[i][0] for i in range(len(resultat5))]
liste_groupes2

['Alkonost', 'Ancient Bards', 'Annatar', 'Arafel', 'Altaria']

In [67]:
common2=[i for i in liste_groupes for j in liste_groupes2 if i==j]
common2

['Altaria', 'Ancient Bards']

# Bilan sur les modèles : Confrontation des résultats obtenus

# Résultat TFIDF Vectorizer

In [68]:
resultat1,resultat2,liste_commune

((['Alkonost', 'Ancient Bards', 'Annatar', 'Arafel', 'Altaria', 'Ajattara'],
  [0.5020352283304641,
   0.46742137155484487,
   0.46742137155484487,
   0.31573363302221696,
   0.3146627358452062,
   0.30328307621846273]),
 (['Arcana',
   'Altaria',
   'Ancient Bards',
   'Alestorm',
   'Almah',
   'Andre Matos',
   'Anvil',
   'Ace Frehley',
   'AC-DC',
   'Airbourne',
   'Aerosmith',
   'Alice Cooper',
   'Angra'],
  [0.49748668349693803,
   0.4879433263597296,
   0.37057893138467557,
   0.36554690540170676,
   0.36554690540170676,
   0.36006527944662303,
   0.36006527944662303,
   0.3241903178647788,
   0.3241903178647788,
   0.3241903178647788,
   0.3241903178647788,
   0.3241903178647788,
   0.3209624107350458]),
 ['Ancient Bards', 'Altaria'])

# Résultats CountVectorizer

In [69]:
resultat3,resultat4,commun

([('Alkonost', 0.6666666666666669),
  ('Ancient Bards', 0.5773502691896258),
  ('Annatar', 0.5773502691896258),
  ('Arafel', 0.4364357804719848),
  ('Aktarum', 0.408248290463863),
  ('Altaria', 0.408248290463863),
  ('Adorned Brood', 0.2886751345948129),
  ('Ajattara', 0.2886751345948129),
  ('Aldaaron', 0.2886751345948129),
  ('Angel Witch', 0.25819888974716115)],
 [('Altaria', 0.6708203932499369),
  ('Alestorm', 0.5773502691896258),
  ('Almah', 0.5773502691896258),
  ('Ancient Bards', 0.5773502691896258),
  ('Andre Matos', 0.5773502691896258),
  ('Angra', 0.5773502691896258),
  ('Anvil', 0.5773502691896258),
  ('Adrenaline Mob', 0.5),
  ('Agalloch', 0.5),
  ('Agent Steel', 0.5)],
 ['Ancient Bards', 'Altaria'])

# Résultats KNN

In [70]:
resultat5,resultat6,common2

([('Alkonost', 0.9979626963664883),
  ('Ancient Bards', 1.0320645604274523),
  ('Annatar', 1.0320645604274523),
  ('Arafel', 1.1698430381703206),
  ('Altaria', 1.1707580998265985)],
 [('Arcana', 1.002510166036297),
  ('Altaria', 1.0119848552624395),
  ('Ancient Bards', 1.121981344421844),
  ('Almah', 1.1264573623518053),
  ('Alestorm', 1.1264573623518053)],
 ['Altaria', 'Ancient Bards'])

In [71]:
liste_groupes,liste3

(['Arcana', 'Altaria', 'Ancient Bards', 'Almah', 'Alestorm'],
 [0.8152842101821494,
  0.8089721979423965,
  0.8089721979423965,
  0.7834703826652962,
  0.7833010112677613])

In [72]:
liste_groupes2,liste4

(['Alkonost', 'Ancient Bards', 'Annatar', 'Arafel', 'Altaria'],
 [0.8154700240463533,
  0.8137260375669133,
  0.7934792109638248,
  0.7926553195869926,
  0.7926553195869926])

# Conclusion avant test corrélation

Au vu des résultats obtenus, et de la facilité d'implémentation de l'algorithme, on en déduit que l'algorithme le plus efficace est le CountVectorizer associé à la cosine_similarity. 

## Count Vectorizer avec corrélation
Dans cette partie, on reprend l'algorithme le plus efficace que l'on a déterminé à l'aide des tests précédents. Néanmoins, dans cette partie, on tente d'utiliser une autre mesure de similarité entre les vecteurs : la corrélation de Pearson, pour voir si on arrive à obtenir de meilleurs résultats qu'avec la similarité cosinus. 

# Colonne paroles

In [73]:
# Reprise de la copie du DataFrame lyrics utilisé dans les parties précédentes. 

lyrics2

,nom,lyrical themes
0,1349,anti religion apocalypse darkness
1,Aäkon Këëtrëh,despair loneliness winter feel
2,Aaskereia,darkness winter forest
3,Abbath,war winter
4,Abigail Williams,metaphysics spiritualism occultism nature
...,...,...
76,Anvil,love heavy metal sex life
77,Aosoth,satanism death darkness
78,Apocalyptica,mostly instrumental love relationship issue
79,Arafel,legends fantasy mysticism abstract theme histo...


In [74]:
#Instanciation de l'algorithme et application aux données comme dans les cas précédents. 

vectorizer = CountVectorizer()

matrix = vectorizer.fit_transform(lyrics2["lyrical themes"])

counts = pd.DataFrame(matrix.toarray(),
                      columns=vectorizer.get_feature_names())

In [75]:
"""
Transposition de la matrice issue de l'algorithme et obtention de la matrice de corrélation entre
tous les vecteurs représentants les thèmes des paroles pour les différents groupes. 
"""

counts=counts.T
corrmatrix=counts.corr()
corrmatrix

,0,1,2,3,4,5,6,7,8,9,...,71,72,73,74,75,76,77,78,79,80
0,1.000000,-0.024845,0.273483,-0.017460,-0.024845,0.273483,-0.033177,-0.024845,-0.024845,0.179888,...,-0.033177,-0.027864,0.273483,-0.027864,0.179888,-0.027864,0.273483,-0.027864,-0.033177,-0.024845
1,-0.024845,1.000000,0.273483,0.342648,-0.024845,-0.021450,-0.033177,-0.024845,-0.024845,-0.030619,...,-0.033177,-0.027864,-0.021450,-0.027864,-0.030619,-0.027864,-0.021450,-0.027864,-0.033177,-0.024845
2,0.273483,0.273483,1.000000,0.399457,-0.021450,0.320988,-0.028643,-0.021450,-0.021450,0.215886,...,-0.028643,-0.024056,-0.018519,-0.024056,-0.026435,-0.024056,0.320988,-0.024056,-0.028643,-0.021450
3,-0.017460,0.342648,0.399457,1.000000,-0.017460,-0.015074,-0.023315,-0.017460,-0.017460,-0.021518,...,-0.023315,0.303513,-0.015074,-0.019582,-0.021518,-0.019582,-0.015074,-0.019582,-0.023315,-0.017460
4,-0.024845,-0.024845,-0.021450,-0.017460,1.000000,-0.021450,-0.033177,-0.024845,0.487578,-0.030619,...,-0.033177,-0.027864,-0.021450,-0.027864,-0.030619,-0.027864,-0.021450,-0.027864,-0.033177,-0.024845
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,-0.027864,-0.027864,-0.024056,-0.019582,-0.027864,-0.024056,-0.037209,-0.027864,-0.027864,-0.034340,...,-0.037209,-0.031250,-0.024056,-0.031250,-0.034340,1.000000,-0.024056,0.175000,-0.037209,0.202013
77,0.273483,-0.021450,0.320988,-0.015074,-0.021450,1.000000,-0.028643,0.273483,0.273483,0.700529,...,-0.028643,0.240563,-0.018519,-0.024056,-0.026435,-0.024056,1.000000,-0.024056,-0.028643,-0.021450
78,-0.027864,-0.027864,-0.024056,-0.019582,-0.027864,-0.024056,-0.037209,-0.027864,-0.027864,-0.034340,...,-0.037209,-0.031250,-0.024056,-0.031250,0.154531,0.175000,-0.024056,1.000000,-0.037209,0.202013
79,-0.033177,-0.033177,-0.028643,-0.023315,-0.033177,-0.028643,-0.044304,-0.033177,-0.033177,-0.040888,...,-0.044304,-0.037209,-0.028643,-0.037209,-0.040888,-0.037209,-0.028643,-0.037209,1.000000,-0.033177


In [76]:
"""
Création d'une fonction retournant les meilleures recommandations pour un groupe donné. Pour cela, on récupère
l'index du groupe recherché pour le trouver dans la matrice de corrélation. On classe les corrélations 
pour ce groupe dans l'ordre décroissant, en éliminant le premier
score qui correspond au groupe recherché. Puis on récupère les index des meilleurs résultats dans une liste
puis, selon le nombre de recommandations, on crée une seconde liste pour ne récupérer que les index qui nous
intéressent. Enfin, en associant les index obtenus pour les scores et ceux du DataFrame, on crée un dictionnaire
qui associe chaque groupe à son score de corrélation avec le groupe de départ. Puis on renvoie ce dictionnaire. 

"""

def recommandation_corr(elem,nbre_de_reco):
    err=lyrics2[lyrics2["nom"]==elem].index[0]
    elem2=corrmatrix[err]
    score=sorted(elem2,reverse=True)
    score=score[1:]
    indexes=[]
    for i in range(len(score)):
        result = np.where(elem2==score[i])
        indexes.append(result)
    indexes=[indexes[i][0][0] for i in range(len(indexes))]
    indexes=indexes[:nbre_de_reco+1]
    indexes=list(dict.fromkeys(indexes)) #Enlève les doubons de la liste. 
    groupes=[lyrics2["nom"][indexes[i]] for i in range(len(indexes))]
    results={groupes[i]:score[j] for i in range(len(groupes)) for j in range(len(score)) if i==j}
    return results


In [77]:
#Affichage des résultats avec le groupe test. 

elem="Amberian Dawn"
n=10
res=recommandation_corr(elem,n)
res

{'Alkonost': 0.6604938271604944,
 'Ancient Bards': 0.5738190417570063,
 'Arafel': 0.5738190417570063,
 'Aktarum': 0.42146622885370394,
 'Altaria': 0.3994571412874183,
 'Aldaaron': 0.39945714128741827,
 'Ajattara': 0.27348301713731177,
 'Adorned Brood': 0.27348301713731094,
 'Angel Witch': 0.2734830171373108,
 'Ancient Rites': 0.24056261216234356}

## Colonne genre
On applique la même méthode que la partie précédente. 


In [78]:
#Récupération de la copie du DataFrame Genre utilisé dans les parties précédentes. 
genre2

,nom,genre
0,12 Stones,metal alternatif
1,12012,visual kei
2,1349,black metal
3,25 Ta Life,hardcore
4,30 Seconds To Mars,metal alternatif
...,...,...
118,Aosoth,black metal
119,Apocalyptica,symphonic heavy metal rock
120,AqME,metal alternatif
121,Arafel,folk black metal


In [79]:
vectorizer = CountVectorizer()

matrix = vectorizer.fit_transform(genre2["genre"])

counts = pd.DataFrame(matrix.toarray(),
                      columns=vectorizer.get_feature_names())
counts=counts.T
corrmatrix=counts.corr()
corrmatrix

,0,1,2,3,4,5,6,7,8,9,...,113,114,115,116,117,118,119,120,121,122
0,1.000000,-0.034483,0.482759,-0.024175,1.000000,-0.024175,-0.034483,-0.024175,1.000000,-0.024175,...,0.513884,-0.024175,-0.034483,0.482759,0.383413,0.482759,0.322590,1.000000,0.383413,-0.034483
1,-0.034483,1.000000,-0.034483,-0.024175,-0.034483,-0.024175,-0.034483,-0.024175,-0.034483,-0.024175,...,-0.046717,-0.024175,1.000000,-0.034483,-0.042601,-0.034483,-0.049629,-0.034483,-0.042601,-0.034483
2,0.482759,-0.034483,1.000000,-0.024175,0.482759,-0.024175,-0.034483,-0.024175,0.482759,-0.024175,...,0.513884,-0.024175,-0.034483,0.482759,0.383413,1.000000,0.322590,0.482759,0.809427,-0.034483
3,-0.024175,-0.024175,-0.024175,1.000000,-0.024175,-0.016949,0.701089,-0.016949,-0.024175,-0.016949,...,-0.032753,1.000000,-0.024175,-0.024175,-0.029867,-0.024175,-0.034794,-0.024175,-0.029867,-0.024175
4,1.000000,-0.034483,0.482759,-0.024175,1.000000,-0.024175,-0.034483,-0.024175,1.000000,-0.024175,...,0.513884,-0.024175,-0.034483,0.482759,0.383413,0.482759,0.322590,1.000000,0.383413,-0.034483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,0.482759,-0.034483,1.000000,-0.024175,0.482759,-0.024175,-0.034483,-0.024175,0.482759,-0.024175,...,0.513884,-0.024175,-0.034483,0.482759,0.383413,1.000000,0.322590,0.482759,0.809427,-0.034483
119,0.322590,-0.049629,0.322590,-0.034794,0.322590,-0.034794,-0.049629,-0.034794,0.322590,-0.034794,...,0.336184,-0.034794,-0.049629,0.322590,0.551825,0.322590,1.000000,0.322590,0.245256,-0.049629
120,1.000000,-0.034483,0.482759,-0.024175,1.000000,-0.024175,-0.034483,-0.024175,1.000000,-0.024175,...,0.513884,-0.024175,-0.034483,0.482759,0.383413,0.482759,0.322590,1.000000,0.383413,-0.034483
121,0.383413,-0.042601,0.809427,-0.029867,0.383413,-0.029867,-0.042601,-0.029867,0.383413,-0.029867,...,0.404011,-0.029867,-0.042601,0.383413,0.298246,0.809427,0.245256,0.383413,1.000000,-0.042601


In [80]:
def recommandation_corr2(elem,nbre_de_reco):
    err=genre2[genre2["nom"]==elem].index[0]
    elem2=corrmatrix[err]
    score=sorted(elem2,reverse=True)
    score=score[1:]
    indexes=[]
    for i in range(len(score)):
        result = np.where(elem2==score[i])
        indexes.append(result)
    indexes=[indexes[i][0][0] for i in range(len(indexes))]
    indexes=list(dict.fromkeys(indexes))#Enlève les doubons de la liste. 
    indexes=indexes[:nbre_de_reco]
    groupes=[genre2["nom"][indexes[i]] for i in range(len(indexes))]
    results={groupes[i]:score[j] for i in range(len(groupes)) for j in range(len(score)) if i==j}
    return results


In [81]:
elem="Amberian Dawn"
n=10
res2=recommandation_corr2(elem,n)
res2

{'Altaria': 0.644658371220304,
 'Alestorm': 0.55182540553647,
 'Ancient Bards': 0.5518254055364699,
 'Almah': 0.5518254055364699,
 'Adrenaline Mob': 0.5518254055364699,
 'Apocalyptica': 0.5518254055364699,
 'Agent Steel': 0.5518254055364697,
 'Agalloch': 0.4642857142857149,
 'Amoral': 0.46428571428571475,
 'Abruptum': 0.4642857142857147}

In [82]:
genre2[genre2["nom"]=='Altaria']["genre"]

78    melodic power metal hard rock
Name: genre, dtype: object

In [83]:
genre2[genre2["nom"]=='Amberian Dawn']["genre"]

82    neoclassical power metal rock
Name: genre, dtype: object

# Conclusion

En comparant les résultats obtenus entre la corrélation et la cosine similarity, on constate que les résultats sont assez similaires. Par conséquent, vu que la cosine_similarity est plus simple à implanter, on en déduit le choix de notre modèle définitif à utiliser pour programmer notre algorithme de recommandation : Le CountVectorizer associé à la cosine_similarity.